In [1]:
%run basic.ipynb

# General Utilities 

This will be included in all the files as generic utilities

In [ ]:
%%writefile  ~/bin/gen/lstmutils1.py
#!/usr/local/bin/python 
'''
Some utility Functions to be used in all the apps
#=*** NOTE *** DO NOT EDIT THIS FILE - THIS iS CREATED FROM: 01_utils.ipynb
'''
import re, sklearn, sys, os, datetime, glob, argparse, json, base64, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16, 4)
mpl.rcParams['axes.grid'] = False

pd.options.display.max_rows = 8

sys.path.append("~/bin/gen")
import ccallbacks

def getconfig(cf = "config*"):
    confFiles = sorted(glob.glob(cf))
    if ( len(confFiles) <= 0):
        print(f"No Configuration files {cf} found!!!")
        sys.exit(1)

    # Read and merge the configuration files
    ret = {}
    for cf in confFiles:
        print(f"#Getting Configuration from {cf}")
        with open(cf, "r") as f:
            cf = f.read()

        if (not cf.find('[START]') >=0 ):
            r1 = cf
        else:
            r1=re.findall("\[START](.*)\[END]", cf, flags=re.MULTILINE|re.DOTALL)
            if ( len(r1) <= 0):
                print(f"Ignoring: Configuration not found in {cf}! no worries")
                continue
            r1 = r1[0].replace("'", '"')    
        rj = eval(r1)
        ret.update(rj)
                
    return ret
    
#-----------------------------------------------------------------------------------
def getConfigList(conf, key=""):
    #print(f"Getting {key}")
    ll = conf.get(key, []);
    ret = []
    for l in ll:
        if type(l) == list:
            ret += l
        elif l.startswith("$"):
            ret += getConfigList(conf, l[1:])
        else:
            ret.append(l)

    return ret
#-----------------------------------------------------------------------------------
def getConfigObject(conf, key=""):
    ll = conf.get(key, "");
    if not ll:
        return;
    
    sst = ll[0]
    dec = base64.b64decode(sst)
    ret = pickle.loads(dec, fix_imports=True)

    return ret
#--------------------------------------------------------------------------------
def runMethod(pyMethod, **kwargs):
    spl = pyMethod.split('.');

    assert len(spl) >= 2, "Hmmm ... May be not what is intended!! module name"

    modName = ".".join(spl[:-1])
    __import__(modName, fromlist="dummy")

    funName = spl[-1]
    ret = None
    for v in sys.modules:
        if (v.startswith(modName)):
            method= getattr(sys.modules[v], funName)
            print(v, type(v), funName, method, type(method), callable(method))
            ret = method(**kwargs)
    return ret


#--------------------------------------------------------------------------------
def getInvertedPreds(conf, yh):
    scalerY = utils1.getConfigObject(conf, "scalerYString")
    sOuputs = utils1.getConfigList(conf, 'scaleOutputs')
    ouputs  = utils1.getConfigList(conf, 'outputs')
    
    yhdf    = pd.DataFrame(yh, columns=ouputs)    # Dataframe of Predictions
    ys      = yhdf[sOuputs].values                # Values to be scaledback
    yi      = scalerY.inverse_transform(ys)       # inverse transform the outputs
    yidf    = yhdf.copy()                         # Copy and set the values
    yidf[sOuputs] = yi
    return yhdf, yidf
    
#--------------------------------------------------------------------------------
def getOriginal(conf, unnormdf, index=0):
    inputs  = utils1.getConfigList(conf, 'inputs')
    ouputs  = utils1.getConfigList(conf, 'outputs')
    tsParams= conf.get("tsParams", {})
    
    index   = 0
    startIX = index + tsParams['length']
    batchSZ = 1 # batch size
    stride  = tsParams.get('stride', 1)
    i = startIX + batchSZ * stride * index
    return unnormdf[i:], inputs, ouputs

#--------------------------------------------------------------------------------
def plotInverted(conf, yh, unnormdf, s =-400, howmany=100):
    e=s+howmany
    
    yhdf, yidf = getInvertedPreds(conf, yh)
    yorg,ips,ops = getOriginal(conf, unnormdf)

    x = pd.to_datetime(yorg[yorg.columns[0]][s:e])
    
    plt.plot(x, yorg[ops].values[s:e], marker='.', label="Original")
    plt.plot(x, yidf.values[s:e], marker='x', label="Predicted")
    plt.title("Plotting Inverted Values:")
    plt.grid()
    plt.legend()

'''
Reconstruct the original diffed columns
'''    
def reconstructOrig(conf, unnormdf, yh, ouputs):
    yhdf, yidf = getInvertedPreds(conf, yh)
    yorg,ips,ops = getOriginal(conf, unnormdf)
    
    for o in ouputs:
        if(o.endswith("___diff1")):
            oc = o[:-8]
            print(f"Getting Original column for: '{oc}' ")
            if ( oc not in yorg.columns):
                print('Cannot compute the orginal column values from diffs for {oc}')
                continue;
                
            ## WOW <== this is heavy - undo the diffing in the opposite way
            yidf[oc] = yorg[oc].values + yidf[o].shift(-1)

    return yidf # y inverted dataframe with adjusted cols for diffs



#runMethod("gen.utils1.is_number", **{"s":"123.78"})

In [ ]:
%%writefile  ~/bin/plotutils1.py
#!/usr/local/bin/python 

import matplotlib.pyplot as plt
import re, sys, os, datetime, glob, json, base64, pickle, sklearn
import pandas as pd
import numpy as np

import keras
from keras.models import Model
from keras.models import load_model
from keras.callbacks import Callback
import IPython
from IPython.display import display
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16, 3)
mpl.rcParams['axes.grid'] = True

sys.path.append(".")
sys.path.append("gen")
sys.path.append("/opt/LMCO/git/bin/gen")
import lstmutils1;
import lstmfit;
import sklearn.metrics

def predict(modelFile, valg, model=None):
    m1 = model or load_model(modelFile)
    #xxt = np.array([valg[i][0][0] for i in range(len(valg))])
    #yyt = np.array([valg[i][1][0] for i in range(len(valg))])
    yh=m1.predict(valg)
    return yh
            
def plot1(modelFile, valg, model=None, idx=0, n=-150, howmany=50):
    yh = predict(modelFile, valg, model)
    yy = np.array([valg[i][1][idx] for i in range(len(valg))])
                 
    plt.gcf().set_size_inches(22, 10, forward=True)
    plt.plot( yy[n:n+howmany], marker='o', label="original-")
    plt.plot( yh[n:n+howmany], marker='x', label="predicted")
    mse = sklearn.metrics.mean_squared_error(yy, yh)
    
    plt.title(f"{modelFile} : {model}: MSE: {mse} <==")
    plt.grid()
    plt.legend()
    plt.show()
    
    return yy, yh, mse


def plotyyyh(columns, xxx, yy1, yh1, s, e):
    for i in range(yh1.shape[-1]):
        yyy = yy1[:,i]
        yyh = yh1[:,i]

        r2 = sklearn.metrics.r2_score(yyy, yyh)
        if (r2 <0.5):
            print(f'--{i}: {columns[i+1]} R^2: {r2}')
            continue

        plt.title(f'{i}: {columns[i+1]} R^2: {r2}')

        plt.plot(xxx[s:e], yyy[s:e], marker='.', label="original")
        plt.plot(xxx[s:e], yyh[s:e], marker='x', label="predictd")
        plt.grid()
        plt.legend()
        plt.show()
    
def plotstuff(valg, confile, normeddf=None, model = None, mcpoint=None ):
    if ( model is None):
        conf, unnormdf, normeddf, inps, oups = lstmfit.getConf(confile)
        model, mcpoint = lstmfit.getModel(conf)

    print(len(valg), mcpoint.best)
    
    xxx = pd.to_datetime(normeddf.time[-len(valg):])
    mcpoint.drawLosses()

    yh1 = model.predict(valg)            
    yy1 = np.array([valg[j][1][0] for j in range(len(valg))])

    s = 0
    e = s + len(valg)
    plotyyyh(normeddf.columns, xxx, yy1, yh1, s, e)
    
    return model, mcpoint, normeddf, yy1, yh1

# Data Configure  - dataprep.py

In [25]:
%%writefile  /opt/utils/gen/dataprep.py
#!/usr/local/bin/python 

# DO NOT EDIT THIS FILE - GENERATED FROM: sada/NNBook/notebooks/NNetworks/LSTM/01_Utils.ipynb

import re, sys, os, datetime, getopt, glob, argparse, datetime, json, base64, pickle,colabexts
import numpy as np
import pandas as pd
from collections import defaultdict
import sklearn.preprocessing
from sklearn.preprocessing import *

#--------------------------------------------------------------------------------
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
#--------------------------------------------------------------------------------
def runMethod(pyMethod, **kwargs):
    spl = pyMethod.split('.');

    assert len(spl) >= 2, "Hmmm ... May be not what is intended!! module name"

    modName = ".".join(spl[:-1])
    __import__(modName, fromlist="dummy")

    funName = spl[-1]
    ret = None
    for v in sys.modules:
        if (v.startswith(modName)):
            method= getattr(sys.modules[v], funName)
            print(v, type(v), funName, method, type(method), callable(method))
            ret = method(**kwargs)
    return ret

#------ 1.--------------------------------------------------------------------------
def dfselect(df, columns=[], **kwargs):
    if not columns or len(columns) <= 0:
        return df
    df1 = df[columns]
    
    return df1
#----- 2.---------------------------------------------------------------------------
def drop_nonnumerics(df, **kwargs):
    #cols = df.columns[df.dtypes.eq('object')]
    cols = df.select_dtypes(exclude=np.number).columns
    cols = [c for c in cols if not c.lower().startswith('time')]
    if (len(cols) > 0):
        print(f"WARNING: *** Non numeric columns => {cols}: Dropping them")
        df.drop(cols, inplace=True, errors="ignore")
    else:
        print(f"+Good: no nonnumeric columns")
        
    return df
#----- 3.---------------------------------------------------------------------------
# Drop any columns with less than 6 unique values
#
def drop_unique(df, unique=6, **kwargs):
    unique_vals  = df.nunique()
    cols         = unique_vals[ unique_vals <= unique].index

    if (len(cols) > 0):
        print(f"WARNING: *** dropping columns having <= {unique} values => {cols}")
        df.drop(cols, inplace=True, errors="ignore")
    else:
        print(f"+Good: Nothing to drop")
        
    return df
    
#----- 4a.---------------------------------------------------------------------------
# Find any sensor highly correlated with time and drop them.
def detectTimeCorrelated(df, timecol="time", val=0.94, **kwargs):
    timecol = df.columns[0]
    
    timeser = pd.Series(df[[timecol]].values.reshape(-1))
    if ( timeser.dtype != np.number ):
        timeser = pd.to_datetime(timeser).astype(int)
    
    
    DROP_INDEX = 0; # Debugging
    corcols    = []
    for sensor in df.columns:
        if (sensor == timecol ):
            continue;
        #print(f"#Testing {sensor}...")
        # The following code tries to detect correlation by dropping first 8 or last 8 values
        # sometimes dropping first few will show correlation due to start up times
        sensorSeries = pd.Series(df[sensor].values.reshape(-1))
        for i in range(8):
            c1 = timeser[i:].corr(sensorSeries[i:])
            c2 = timeser[i:].corr(sensorSeries[:-i])
            if np.abs(c1) >= val or np.abs(c2) >= val:
                corcols.append(sensor)
                DROP_INDEX = max(DROP_INDEX, i) #lets drop first few rows
                break;
                
    #print(f"#Time Cor: #{len(timeCorSensors)}, #Shape before:{df.shape}")
    #df.drop(timeCorSensors, axis=1, inplace=True)
    #df = df[DROP_INDEX:]
    #print(f"#After dropping: {DROP_INDEX} =>{df.shape}")
        
    return corcols
#----- 3.---------------------------------------------------------------------------
# Drop any time correlated sensors
#
def drop_time_correlated(df, timecol="time", corr=0.95, **kwargs):
    cols = detectTimeCorrelated(df, timecol, corr)

    if (len(cols) > 0):
        print(f"WARNING: Dropping time correlated columns having corr-coeffient >= {corr} => {cols}")
        df.drop(cols, axis=1, inplace=True, errors="ignore")
    else:
        print(f"+Good: No time correlated columns having corr-coeffient >= {corr}")
        
    return df
#-----------------------------------------------------------------------------------
# Covert to one_hot encoding with prefix for columns'
def make_OneHot(df, oheCols=[], **kwargs):
    ohe = pd.DataFrame();
    for c in oheCols:
        one_hot = pd.get_dummies(df[c])
        nc = [f'{c}___{k}' for k in one_hot.columns]
        one_hot.columns = nc
        ohe = pd.concat([ohe, one_hot], axis=1)

    return ohe
#-----------------------------------------------------------------------------------
# Assuming the tf1 is sorted in ascending order of time
def add_Diff(tf1, col=[], **kwargs):
    if (type(col) == str):
        col = [col]
    for c in col:
        if ( c not in tf1.columns):
            print(f"*WARNING* Column {c} Not FOUND")
            continue
        print(f"+++ Adding {c}")
        tf1[f'{c}___diff1'] = tf1[c] - tf1[c].shift(1)
    return tf1
#-----------------------------------------------------------------------------------
# Assuming the tf1 is sorted in ascending order of time
# column1 = 'MSFT_close'
# column2 = 'AAPL_close'
def add_corr(df, column1, column2, window  = 100, stride=1, **kwargs):
    c1      = df[column1]
    c2      = df[column2]
    corr    = []
    for i in range(0, len(c1) - window +1, stride):
        cc1 = c1[i : i+window]
        cc2 = c2[i : i+window]
        cor = cc1.corr( cc2 )
        corr.append(cor)
    df = df[window:]    
    df[column1+"___"+column2] = corr
    return df
#-----------------------------------------------------------------------------------
def _convertObjToStr(obj):
    astr = base64.b64encode(pickle.dumps(obj, protocol=None, fix_imports=True))
    astr = astr.decode("utf-8")
    return astr
def _convertStrToObj(astr):
    astr = base64.b64decode(astr)
    obj  = pickle.loads(decoded,fix_imports=True)
    return obj
#-----------------------------------------------------------------------------------
def add_movingavg(df, cols = [], window=0, dropna=True, **kwargs):
    for c in cols:
        a = df[c].rolling(window=window).mean()
        df[f'{c}__MOVING_AVG'] = a
    if (dropna):
        df.dropna(inplace=True)
    #uni_data.plot(subplots=True)
    #u3.plot(subplots=True, color='red')
    return df;

def add_expmovingavg(df, cols = [], window=0, dropna=True, **kwargs):
    for c in cols:
        a = df[c].ewm(span=window,adjust=False).mean()
        df[f'{c}__EXP_MOVING_AVG'] = a
    if (dropna):
        df.dropna(inplace=True)
        
    return df

def data_fill(df,  **kwargs):
    df = df.ffill().bfill()
    df.dropna(inplace=True)
        
    return df
#-----------------------------------------------------------------------------------
# This will scale the numeric columns - if this changes - you need to use dataprep
#    "scaler"         : ["sklearn.preprocessing.StandardScaler()"],
#    "scaler"         : ["sklearn.preprocessing.MinMaxScaler()"],
def data_scale(df, cols = [], start=0, pct=0.9, count=0, scaler=StandardScaler, **kwargs):
    if ( type(scaler) == type ):
        scaleri = scaler()
    elif ( type(scaler) == str ):
        scaleri = _convertStrToObj(scaler)
    else:
        scaleri = scaler
    
    
    if isinstance(df, pd.DataFrame):
        cols = cols or [c for c in df.columns] 
        df1  = df[cols]
    else:
        df1  = df
        
    tCnt = count or int(len(df) * pct) or None
        
    sx1  = scaleri.fit(df1[start:tCnt])        # Fit only training part 
    di1  = scaleri.transform(df1)              # Tranform the entire df
    dfi  = pd.DataFrame(di1, columns=cols )

    return dfi, _convertObjToStr(scaleri);
#-----------------------------------------------------------------------------------
def process():
    print("Dont know what to do!! NOW")
    
#-----------------------------------------------------------------------------------
sysargs=None
def addargs():
    global sysargs
    p = argparse.ArgumentParser(f"{os.path.basename(sys.argv[0])}:")
    p.add_argument('-c', '--config', type=str, default="config.txt", help="Config Files")
    p.add_argument('-o', '--output', type=str, default=0, help="output file")
    p.add_argument('args', nargs=argparse.REMAINDER)
    p.add_argument('input_files',action="store", type=str, nargs='+', help="input file(s)")

    try:
        sysargs, unknown=p.parse_known_args(sys.argv[1:])
    except argparse.ArgumentError as exc:
        print(exc.message )
        
    if (unknown):
        print("Unknown options: ", unknown)
        #p.print_help()
    return sysargs    
#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not colabexts.jcommon.inJupyter()):
        t1 = datetime.datetime.now()
        sysargs = addargs()
        ret = process()
        t2 = datetime.datetime.now()
        print(f"#All Done in {str(t2-t1)} ***")
    else:
        pass


Overwriting /opt/utils/gen/dataprep.py


In [42]:
a=[
    [1,2,3],
    [11,21,33],
    [12,22,32],
    [13,23,33],
]
a1=np.array(a)
a1, a1[:,:-1]

(array([[ 1,  2,  3],
        [11, 21, 33],
        [12, 22, 32],
        [13, 23, 33]]),
 array([[ 1,  2],
        [11, 21],
        [12, 22],
        [13, 23]]))

In [39]:
a1[:,2]

array([ 3, 33, 32, 33])

# OLD

In [ ]:
%%writefile  ~/bin/gen/dataconfig.py
#!/usr/local/bin/python 

import re, sys, os, glob, argparse, datetime, json

#-----------------------------------------------------------------------------------
'Covert to one_hot encoding with prefix for columns'
def makeOneHotCols(tf1, oheCols=[]):
    ret = []
    for c in oheCols:
        one_hot = pd.get_dummies(tf1[c])
        ret += [f'{c}___{k}' for k in one_hot.columns]

    return ret

#-----------------------------------------------------------------------------------
def detectCols(file, nUnique=4, tcoeff=0.92):
    tf1 = file
    if (type(tf1) == str):
        tf1 = pd.read_csv(tf1, comment="#")
    
    #Lets check if it has any non-numeric columns! Warning
    precheck(tf1)

    unique_vals  = tf1.nunique()
    constantCols = unique_vals[ unique_vals == 1].index                             # constant Columns
    onehotECols  = unique_vals[(unique_vals > 2 ) & (unique_vals<=nUnique)].index   # Categorical Columns
    categorCols  = unique_vals[(unique_vals >=2 ) & (unique_vals <= nUnique)].index # Categorical Columns
    binaryCols   = unique_vals[(unique_vals == 2)].index                      # Binary

    numericCols  = tf1.select_dtypes(include=np.number).columns           # numerics
    numericCols  = [c for c in numericCols if c not in categorCols]
    numericCols  = [c for c in numericCols if c not in constantCols]
    numericCols  = [c for c in numericCols if c != 'time']
    notNumerics  = tf1.select_dtypes(exclude=np.number).columns           # non - numerics
    notNumerics  = [c for c in notNumerics if c not in categorCols]       # non - numerics

    try:
        pass #timeCorCols  = detectTimeCorrelated(tf1, tcoeff)
    except:
        timeCorCols = []
    
    onehotEC_ext = makeOneHotCols(tf1, onehotECols)
   
    ret1 =f'''[START]
{{
    "file"           : {[file] if (type(file) == str) else ["??"]},
    "nrowsXncols"    : {[len(tf1), len(tf1.columns )] }     , 
    "number_Unique"  : {nUnique}            , 
    "constantCols"   : {list(constantCols )},   # No Signals
    "#constantCols"  : {len(constantCols  )},   # No Signals
    "categorCols"    : {list(categorCols  )},   # Categorical Columns
    "#categorCols"   : {len(categorCols   )},   # Categorical Columns
    "onehotECols"    : {list(onehotECols  )},   # Cats > 2 and < Unique Values
    "onehotEC_ext"   : {list(onehotEC_ext )},   # Cats > 2 and < Unique Values
    "#onehotECols"   : {len(onehotECols   )},   # Cats > 2 and < Unique Values
    "binaryCols"     : {list(binaryCols   )},   # Binary
    "#binaryCols"    : {len(binaryCols    )},   # Binary
    "notNumerics"    : {list(notNumerics  )},
    "timeCorrelation": {tcoeff             },   # Time correlated
    "timeCorrCols"   : {list(timeCorCols  )},   # Time correlated Columns
    "#timeCorrCols"  : {len(timeCorCols   )},    # Time correlated Columns
    "excludePattern" : [] , #Exclude patterns
    "includePattern" : [] , #include patterns
    "dropColumns"    : [],
    "diff_suffix"    : {['__diff1']},
    "addDiffs"       : [],
    "train_pct"      : .9,
    "#numericCols"   : {len(numericCols   )},  
    "scaleInputs"    : {list(numericCols  )},  
    "scaleOutputs"   : {["$scaleInputs"]},  
    "inputs"         : {["$binaryCols", "$scaleInputs", "$onehotECols"]},
    "outputs"        : {["$scaleOutputs"]},
#-----Copy this generated file and add customization
    "loadModel"      : 1,
    "scale"          : 1,
    "scaler"         : ["sklearn.preprocessing.StandardScaler()"],
    "scaler"         : ["sklearn.preprocessing.MinMaxScaler()"],
    "scalerXString"  : [],
    "scalerYString"  : [],
    "tsParams"       : {{"length": 60, "batch_size": 1, "stride": 1, "sampling_rate": 1}},
    "lookahead"      : 60,
    "nsteps"         : 1,
    "modelFile"      : ["lstm.56.h5"],
    "monitor"        : "val_loss",
    "modelName"      : "gen.somemodels.SimpleModel1(50, 5, 1, **{{}})"
}}
[END]
    '''
    return ret1, tf1;
#-----------------------------------------------------------------------------------    
def process():
    n  = len(sysargs.input_files)
    un = sysargs.unique
    tc = sysargs.tcoeff
    for i, file1 in enumerate(sysargs.input_files):
        print(f"#=>Processing {i+1}/{n} {file1} #unique: {un} tcoeff: {tc} - standby")
        outs, df = detectCols(file1, un, tc)
        
        break;
    print(outs)
    return outs
    
#-----------------------------------------------------------------------------------
sysargs=None
def addargs():
    sysargs = None
    p = argparse.ArgumentParser(f"{os.path.basename(sys.argv[0])}:")
    p.add_argument('-u', '--unique', type=int,   default=6,    help="# of unique values!")
    p.add_argument('-t', '--tcoeff', type=float, default=0.94, help="# Time Correlation value Sensors!")
    p.add_argument('args', nargs=argparse.REMAINDER)
    p.add_argument('input_files',action="store", type=str, nargs='+', help="input file(s)")

    #p.print_help() # always print -help
    try:
        sysargs=p.parse_args(sys.argv[1:])
        #print(f'using:\n{sysargs}')
    except argparse.ArgumentError as exc:
        #par.print_help()
        print(exc.message )
        
    return sysargs
#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        sysargs = addargs()
        process()
        t2 = datetime.datetime.now()
        print(f"#All Done in {str(t2-t1)} ***")
    else:
        pass

# Data Prepare

In [ ]:
%%writefile  ~/bin/gen/dataprepare.py
#!/usr/local/bin/python 

import re, sklearn, sys, os, datetime, getopt, glob, argparse, datetime, json, base64, pickle
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

sys.path.append(".")
sys.path.append("gen")
sys.path.append("~/bin/gen")
import lstmutils1;

'''
Make sure data is sorted in asceding order of the time for LSTM to work and all these 
data prep tools to work.
'''


'Covert to one_hot encoding with prefix for columns'
def makeOneHot(tf1, oheCols=[]):
    ohe = pd.DataFrame();
    for c in oheCols:
        one_hot = pd.get_dummies(tf1[c])
        nc = [f'{c}___{k}' for k in one_hot.columns]
        one_hot.columns = nc
        ohe = pd.concat([ohe, one_hot], axis=1)

    return ohe

'''
Assuming the tf1 is sorted in ascending order of time
'''
def addDiff(tf1, col):
    #col = "MSFT_open"
    if (type(col) == str):
        col = [col]
    for c in col:
        if ( c not in tf1.columns):
            print(f"*WARNING* Column {c} Not FOUND")
            continue
        print(f"+++ Adding {c}")
        tf1[f'{c}___diff1'] = tf1[c] - tf1[c].shift(1)
    return tf1

#-----------------------------------------------------------------------------------
def formatConfig(out: dict):
    outj = f"[START]\n{{\n"
    for k,v in out.items():
        kk = f'"{k}"'
        vv = f"'{v}'" if type(v) == str else v
            
        outj += f'{kk:>20}: {vv},\n'
    outj += '"end": 0 \n}\n[END]\n'
    
    return outj

#-----------------------------------------------------------------------------------
def getFinalColumns(df, conf):
    inputCols = lstmutils1.getConfigList(conf, "inputs")
    ouputCols = lstmutils1.getConfigList(conf, "outputs")

    allcols = set(inputCols +ouputCols);
    
    assert allcols.issubset(set(df.columns)), "Hmmm ... columns missing"
    return sorted(list(allcols))
#-----------------------------------------------------------------------------------
'''
This will scale the numeric columns - if this changes - you need to use dataprep
'''
def scaleNumerics(df, conf={}):
    scaleInputCols = lstmutils1.getConfigList(conf, 'scaleInputs')
    scaleOuputCols = lstmutils1.getConfigList(conf, 'scaleOutputs')
    
    dfninps = df[ scaleInputCols ]
    dfnouts = df[ scaleOuputCols ]

    scale   = conf.get('scale', 0)
    dfninpsn, dfnoutsn, scalerX, scalerY = dfninps, dfnouts, None, None
    
    if (scale):
        scalerXStr = conf['scalerXString']
        scalerYStr = conf['scalerYString']
        trnPct     = conf.get('train_pct', 0.9);
        trnCnt     = int(len(df) * trnPct)

        conf["train_pct"   ] = trnPct
        conf["train_count" ] = trnCnt

        if (not scalerXStr):
            scaler  = conf.get('scaler', ["sklearn.preprocessing.MinMaxScaler()"]);
            scalerX = eval(scaler[0]) if type(scaler[0]) == str else scaler
            scalerX = scalerX.fit(dfninps[:trnCnt])
            scalerstr = base64.b64encode(pickle.dumps(scalerX, protocol=None, fix_imports=True))
            scalerstr = scalerstr.decode("utf-8")
            conf["scalerXString"] = [scalerstr]
            #print(f'==>+1 shape: {dfninps.shape} {scalerX.mean_}')
        else:
            scalerXStr = scalerXStr[0]
            decoded    = base64.b64decode(scalerXStr)
            scalerX    = pickle.loads(decoded,fix_imports=True)

        if (not scalerYStr):
            scaler  = conf.get('scaler', ["sklearn.preprocessing.MinMaxScaler()"]);
            scalerY = eval(scaler[0]) if type(scaler[0]) == str else scaler
            scalerY = scalerY.fit(dfnouts[:trnCnt])
            scalerstr = base64.b64encode(pickle.dumps(scalerY, protocol=None, fix_imports=True))
            scalerstr = scalerstr.decode("utf-8")
            conf["scalerYString"] = [scalerstr]
            #print(f'==>+2 shape: {dfninps.shape} {scalerY.mean_}')
        else:
            scalerYStr = scalerYStr[0]
            decoded    = base64.b64decode(scalerYStr)
            scalerY    = pickle.loads(decoded,fix_imports=True)

        di = scalerX.transform(dfninps)
        do = scalerY.transform(dfnoutsn)
        
        dfninpsn = pd.DataFrame(di, columns=scaleInputCols )
        dfnoutsn = pd.DataFrame(do, columns=scaleOuputCols )
        
        #print(f'==>++ shape: {dfninpsn.shape} {scalerX.mean_}')
        #print(f'==>++ shape: {dfnoutsn.shape} {scalerY.mean_}')

    return dfninpsn, dfnoutsn, scalerX, scalerY;
#-----------------------------------------------------------------------------------
def process(config, input_files, output=None):
    conf = lstmutils1.getconfig(config)
    
    n  = len(input_files)
    adfInp = pd.DataFrame();
    for i, file1 in enumerate(input_files):
        print(f"=>Processing {i+1}/{n} {file1} - standby")
        df = pd.read_csv(file1, comment='#')
        
        drps = lstmutils1.getConfigList(conf, "dropColumns")
        df.drop(drps, inplace=True, errors="ignore")
        
        # STEP 1: Add diffs
        cdiffs  = lstmutils1.getConfigList(conf, 'addDiffs')
        addDiff(df, cdiffs)    #<< 1. Add tis
        df.dropna(inplace=True)
        df.reset_index(inplace=True, drop=True)

        # STEP 2: => One hot encode 
        ohe = None
        ohecols = lstmutils1.getConfigList(conf, 'onehotECols')
        if len(ohecols) > 0:
            ohe=makeOneHot(df, conf['onehotECols'])  #< === ADD
            df=pd.concat([df,ohe], axis=1)
         
        # STEP 3: Add
        allCols = [df.columns[0]] + getFinalColumns(df,conf)
        dfunNormalized = df[allCols]
        #Numeric Columns
        dfiNorm, dfoNorm, sX, sY = scaleNumerics(dfunNormalized, conf)
        
        dfNormalized = dfunNormalized.copy()
        dfNormalized[dfiNorm.columns] = dfiNorm;
        dfNormalized[dfoNorm.columns] = dfoNorm;
        
        #FINALLY        
        if (output is not None):
            fi,ext = os.path.splitext(file1)
            nfu  = f'{os.path.basename(fi)}_Orig_{i}{ext}'
            print(f"writing unnormalized to: {nfu}")
            dfunNormalized.to_csv(nfu, index=False)
            
            
            nfn  = f'{os.path.basename(fi)}_Norm_{i}{ext}'
            print(f"writing normalized to. : {nfn}")
            dfNormalized.to_csv  (nfn, index=False)
            
            conf['normalizedFile']   = nfn
            conf['unnormalizedFile'] = nfu
            
        break;

    outj = formatConfig(conf)
    print(outj)
    
    return conf, dfunNormalized, dfNormalized
#-----------------------------------------------------------------------------------
sysargs=None
def addargs():
    sysargs = None
    p = argparse.ArgumentParser(f"{os.path.basename(sys.argv[0])}:")
    p.add_argument('-c', '--config', type=str, default="config.txt", help="Config Files")
    p.add_argument('-o', '--output', type=str, default=0, help="output file")
    p.add_argument('args', nargs=argparse.REMAINDER)
    p.add_argument('input_files',action="store", type=str, nargs='+', help="input file(s)")

    #p.print_help() # always print -help
    try:
        sysargs=p.parse_args(sys.argv[1:])
    except argparse.ArgumentError as exc:
        print(exc.message )
        
    return sysargs
    
#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not inJupyter()):
        t1 = datetime.datetime.now()
        sysargs = addargs()
        ret = process(sysargs.config, sysargs.input_files, sysargs.output)
        t2 = datetime.datetime.now()
        print(f"#All Done in {str(t2-t1)} ***")
    else:
        pass

In [ ]:
f='/opt/SCHAS/NNBook/notebooks/NNetworks/LSTM/data/stockdata_ext.csv'
f='/opt/SCHAS/NNBook/notebooks/NNetworks/LSTM/data/daily_MSFT.csv'
conf, dfUnNormalized, dfNormalized = process('config.*', [f], "out.csv")

# Custom Callback for Saving and Loading Models

In [ ]:
%%writefile  ~/bin/gen/ccallbacks.py
#!/usr/local/bin/python 

import matplotlib.pyplot as plt
import re, sys, os, datetime, glob, json, base64, pickle, sklearn
import pandas as pd
import numpy as np

import keras
from keras.models import Model
from keras.models import load_model
from keras.callbacks import Callback
import IPython
from IPython.display import display

sys.path.append("gen")
sys.path.append("/opt/LMCO/git/bin/gen")
import lstmutils1;
import sklearn.metrics

class ModelCheckAndLoad(Callback):
    def __init__(self, filepath, monitor='val_loss', best=np.inf, 
                 stop_at=False, verbose=0, drawLoss=False):
        super(Callback, self).__init__()
        self.monitor  = monitor
        self.filepath = filepath
        self.verbose  = verbose
        self.best     = best or np.inf
        self.stop_at  = stop_at;
        self.history  = {}
        self.epochs   = []
        self.drawLoss = drawLoss
        self.epochNum = 0
        self.numSaved = 0
        
    def save_ext(self):
        ef = self.filepath+"_ext"
        with open(ef, "wb") as f:
            myParams = {
                'best'     : self.best,
                'bestEpoch': self.bestEpoch,
                'epochNum' : self.epochNum,
                'history'  : self.history,
                'monitor'  : self.monitor
            }
            pickle.dump(myParams, f, protocol=pickle.HIGHEST_PROTOCOL)
                
    def save_latest(self):
        self.model.save(self.filepath+"_latest", overwrite=True)
        self.save_ext();
            
    def load_ext(self):
        ret = None;
        if ( os.path.exists(self.filepath+"_latest")):
            ret = load_model(self.filepath+"_latest")
            print("Loading from the latest:...")
        elif ( os.path.exists(self.filepath)):
            ret = load_model(self.filepath)
        
        ef = self.filepath+"_ext"
        if ( not os.path.exists(ef) or os.path.getsize(ef) <= 0):
            return ret
        
        with open(ef, "rb") as f:
            myParams      = pickle.load(f)
            self.best     = myParams.get('best'    , np.inf)
            self.epochNum = myParams.get('epochNum', 0);
            self.history  = myParams.get('history', {});
            self.monitor  = myParams.get('monitor'  , "val_loss");
            
        print(f"Best Loaded {self.best} occured at: {self.epochNum}")
        return ret;

    def drawLosses(self):
        history, best = self.history, self.best
        #IPython.display.clear_output(wait=True)
        plt.clf()

        fig, ax1 = plt.subplots()
        i, colors, marks = 0, "rgbcmykw", "v.xo+"

        color = colors[i]
        ax1.set_xlabel('epochs')
        k, v = "loss", history['loss']
        ax1.set_ylabel(k, color=color)
        l1= ax1.plot(v, color=color, marker=marks[i], label=f"{k}")

        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

        i +=1
        k, v = "val_loss", history['val_loss']
        color = colors[i]
        ax2.set_ylabel(k, color=color)
        l2 = ax2.plot(v, color=color, marker=marks[i], label=f"{k}")

        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        l3 = plt.plot(0, best, marker="o",  c="b", label=f"BEST: {best}")
        ax1.grid()

        lns  = l1 + l2 + l3;
        labs = [l.get_label() for l in lns]
        plt.legend(lns, labs, loc=0)
        plt.show()
        
    def on_epoch_end(self, epoch, logs={}):
        self.epochs.append(epoch)
        self.epochNum += 1;
        
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        self.current = logs.get(self.monitor)
        if self.current is None:
            warnings.warn(f'Can save best model only with {self.monitor} available')
            return;
                    
        if (self.best > self.current):
            ou= f'{self.monitor}: {self.best} > {self.current}\n'
            print(f"Epoch: {epoch+1} Saving: {ou}");
            
            self.numSaved += 1
            self.bestEpoch+= 1
            self.best      = self.current
            self.model.save(self.filepath, overwrite=True)
            self.save_ext();
            self.model.stop_training = self.stop_at
        elif self.verbose > 0:
            ou= f'{self.monitor}: {self.best} <= {self.current}'
            print(f"{epoch+1} din't improve : {ou} from {self.bestEpoch}\r", end="")
            
        if (self.drawLoss):
            drawLosses(self.history, self.best)


# Definition of Some Model Architectures

In [ ]:
%%writefile  ~/bin/gen/LSTMModelDefs.py
#!/usr/local/bin/python 

import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras.layers import RepeatVector, TimeDistributed
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import optimizers

def SimpleModel1(history, nfeatures, nOut, **kwargs) :
    lstm_input = Input(shape=(history, nfeatures), name='lstm_input')
    x = LSTM(50, name='lstm_0')(lstm_input)
    x = Dropout(0.2, name='lstm_dropout_0')(x)
    x = Dense(64, name='dense_0')(x)
    x = Activation('sigmoid', name='sigmoid_0')(x)
    x = Dense(nOut, name='dense_1')(x)
    output = Activation('linear', name='linear_output')(x)

    model = Model(inputs=lstm_input, outputs=output)
    adam = optimizers.Adam(lr=0.0005)
    model.compile(optimizer=adam, loss='mse')
    
    return model

def SimpleModel2(inps, inshape, units2=None, nsteps=1, opt="adam", loss="mse", bi=False, dropout=None):
    s= inshape
    print(locals())
    print(f"Creating LSTM: inuts= {inps} time-steps: {s[0]}, features: {s[1]} #out: {nsteps}")
    m = keras.models.Sequential()

    if (bi):
        m.add(keras.layers.Bidirectional(
            keras.layers.LSTM(inps, return_sequences= (units2 is not None), input_shape=s) ) )
    else:
        m.add(keras.layers.LSTM(inps, return_sequences= (units2 is not None), input_shape=s) )
    
    if(units2 is not None): #Lets just keep it simple for 2 layers only
        m.add(keras.layers.LSTM(units2, activation='relu'))
    if (dropout is not None):
        m.add( keras.layers.Dropout(dropout) )
    m.add(keras.layers.Dense(nsteps))
    m.compile(optimizer = opt, loss= loss)
    return m

def UberModel(lookBack, nFeatures, lstm_IPDim=256, lstm_OPDim=1, opt=None, loss="rmse",  drop=0.3):
    opt        = opt or optimizers.Adam(lr=0.0005)
    k_rrizer   = None
    r_rrizer   = None

    input_layer  = Input(shape=(lookBack, nFeatures), dtype='float32', name='input')
    memory_layer = LSTM( lstm_IPDim, return_sequences=True, name="memory1")(input_layer)
    memory_layer = LSTM (int(lstm_IPDim/2), return_sequences=False, name="memory2")(memory_layer)
    repeated     = RepeatVector(lookBack)(memory_layer)
    memory_layer = LSTM (int(lstm_IPDim/2), return_sequences=True, name="first1out")(repeated)
    memory_layer = LSTM (lstm_IPDim,  return_sequences=True, name="first2out")(memory_layer)
    decoded_inputs = TimeDistributed(Dense(units=lstm_OPDim, activation='linear'))( memory_layer)

    #  Try spatial dropout?
    dropout_input = Dropout(drop)(input_layer)
    concat_layer  = concatenate([dropout_input, decoded_inputs])

    memory_layer = LSTM (units=lstm_IPDim, 
                             kernel_regularizer = k_rrizer, 
                             recurrent_regularizer = r_rrizer, 
                             return_sequences=False)(concat_layer)
    preds = Dense(units=lstm_OPDim, activation='linear')(memory_layer)

    model1 = Model(input_layer, preds)
    model1.compile(optimizer = opt, loss= loss)             

    return model1